# HW4 Trees, Forests, and Bag of Words

Official instructions:

https://www.cs.tufts.edu/comp/135/2020f/hw4.html

This is the *starter code* notebook.

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [2]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble

In [3]:
# From the HW4 starter code
from pretty_print_sklearn_tree import pretty_print_sklearn_tree

In [4]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

# Load all data from train/valid/test

In [5]:
# TODO fix to path on your local system
DATA_DIR = '/users/zoehsieh/Desktop/Comp135/hw4/data_product_reviews/'


### Load training

In [6]:
x_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv.zip'))
y_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
x_tr_NF = np.minimum(x_tr_df.values, 1.0).copy()
y_tr_N = y_tr_df.values[:,0].copy()

print("Training data")
print("x_tr_NF.shape: %s" % str(x_tr_NF.shape))
print("y_tr_N.shape : %s" % str(y_tr_N.shape))
print("mean(y_tr_N) : %.3f" % np.mean(y_tr_N))

Training data
x_tr_NF.shape: (6346, 7729)
y_tr_N.shape : (6346,)
mean(y_tr_N) : 0.500


### Load validation set

In [7]:
x_va_df = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv.zip'))
y_va_df = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv'))

x_va_TF = np.minimum(x_va_df.values, 1.0).copy()
y_va_T = y_va_df.values[:,0].copy()

print("Validation data")
print("x_va_TF.shape: %s" % str(x_va_TF.shape))
print("y_va_T.shape : %s" % str(y_va_T.shape))
print("mean(y_va_T) : %.3f" % np.mean(y_va_T))

Validation data
x_va_TF.shape: (792, 7729)
y_va_T.shape : (792,)
mean(y_va_T) : 0.490


### Load test set 

In [8]:
x_te_df = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv.zip'))
y_te_df = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv'))

x_te_TF = np.minimum(x_te_df.values, 1.0).copy()
y_te_T = y_te_df.values[:,0].copy()

print("Heldout Test data")
print("x_te_TF.shape: %s" % str(x_te_TF.shape))
print("y_te_T.shape : %s" % str(y_te_T.shape))
print("mean(y_te_T) : %.3f" % np.mean(y_te_T))

Heldout Test data
x_te_TF.shape: (793, 7729)
y_te_T.shape : (793,)
mean(y_te_T) : 0.515


### Load vocabulary as a list of strings

In [9]:
vocab_list = x_tr_df.columns.tolist()

In [10]:
for word in vocab_list[:8]:
    print(word)
print("...")
for word in vocab_list[-8:]:
    print(word)

good
great
time
book
don't
work
i_have
read
...
never_get
i'd_like
loves_it
an_author
nomin
could_give
bad_but
gap


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [11]:
xall_LF = np.vstack([x_tr_NF, x_va_TF])
yall_L = np.hstack([y_tr_N, y_va_T])

In [12]:
valid_indicators_L = np.hstack([
    -1 * np.ones(y_tr_N.size), # -1 means never include this example in any test split
    0  * np.ones(y_va_T.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])

In [13]:
# Create splitter object using Predefined Split
# Will be used later by all hyperparameter searches

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

# Problem 1: Decision Trees

## 1A: Train a simple tree with depth 3

In [14]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    max_depth=3, min_samples_split=2, min_samples_leaf=1, criterion='gini')

### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [15]:
simple_tree.fit(x_tr_NF, y_tr_N) #TODO

DecisionTreeClassifier(max_depth=3)

### **Print Tree** 

Use a helper function from the starter code

In [16]:
pretty_print_sklearn_tree(simple_tree, feature_names=vocab_list)

The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (4041 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (368 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (277 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (14 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1413 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (142 total training examples)
    N Decision: X['movie'] <= 0.50?
    

## 1B : Find best Decision Tree with grid search

In [17]:
# Construct the default predictor
# Any hyperparameters here may be overridden by the hyperparameter grid

tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', min_samples_split=2, min_samples_leaf=1)

In [18]:
tree_hyperparameter_grid_by_name = dict(
    max_depth=[2, 8, 32, 128],
    min_samples_leaf=[1, 3, 9],
    random_state = [101],
    )

**TODO Build the Grid Search** in the next coding cell

Hint: See Lab for Grid Search: https://github.com/tufts-ml-courses/comp135-20f-assignments/blob/master/labs/day13_HyperparameterSearch.ipynb

Key Function: sklearn.model_selection.GridSearchCV

Key Requirements:

* Provide the above tree_hyperparameter_grid_by_name dictionary as the set of hyperparameters to search
* Set scoring='balanced_accuracy', since our target metric is balanced accuracy
* Set cv=my_splitter so you can use the predefined split we defined earlier.
* Set return_train_score=True (we want training set scores as well as test set scores)
* Set refit=False (we only want fits on x_tr not on x_all)

In [19]:
tree_grid_searcher = sklearn.model_selection.GridSearchCV(
    simple_tree,
    tree_hyperparameter_grid_by_name,
    scoring='balanced_accuracy',
    cv=my_splitter,
    return_train_score=True,
    refit=False)

### Do the search!


In [20]:
start_time_sec = time.time()
tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

### Build dataframe of results

Move the results of grid search into a nice pandas data frame.

In [21]:
tree_search_results_df = pd.DataFrame(tree_grid_searcher.cv_results_).copy()
print("Grid search of %3d configurations done after %6.1f sec" % (
    tree_search_results_df.shape[0], elapsed_time_sec))

Grid search of  12 configurations done after  144.4 sec


### Display search results

This block will make a pretty printed table of the results of your grid search

In [22]:
pd.set_option('precision', 4)
tree_keys = ['param_max_depth', 'param_min_samples_leaf']
tree_search_results_df.sort_values(tree_keys, inplace=True)
tree_search_results_df[tree_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']]

,param_max_depth,param_min_samples_leaf,mean_train_score,mean_test_score,rank_test_score,mean_fit_time
0,2,1,0.6404,0.6339,10,2.9640
1,2,3,0.6404,0.6339,10,2.4456
2,2,9,0.6404,0.6339,10,2.2847
3,8,1,0.7279,0.6989,7,6.7511
4,8,3,0.7232,0.6978,8,7.3906
5,8,9,0.7123,0.6962,9,6.8500
6,32,1,0.9187,0.7325,3,16.2346
7,32,3,0.8752,0.7349,2,15.6196
8,32,9,0.8121,0.7197,6,14.7058
9,128,1,0.9981,0.7366,1,27.5890


In [23]:
print("Printing a dict of the best hyperparameters")
print(tree_grid_searcher.best_params_)

Printing a dict of the best hyperparameters
{'max_depth': 128, 'min_samples_leaf': 1, 'random_state': 101}


### Build the best decision tree

**TODO Build the Best Tree** in the next coding cell

This is necessary so you have the specific best performing tree in your workspace.

Although you fit many trees in the search, they were not stored, so we need to recreate the best one.

Hint: Just feed the best hyperparameters as keyword args to construct the tree. Or see the lab about grid search.

In [24]:
best_tree = tree.set_params(max_depth=128, min_samples_leaf=1, random_state=101) #TODO
best_tree.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(max_depth=128, random_state=101)

### Interpret the best decision tree

In [25]:
best_tree.tree_

In [26]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)

The binary tree structure has 1587 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    0 are leaves
- depth   4 has   16 nodes, of which    5 are leaves
- depth   5 has   22 nodes, of which    6 are leaves
- depth   6 has   32 nodes, of which   13 are leaves
- depth   7 has   38 nodes, of which   21 are leaves
- depth   8 has   34 nodes, of which   16 are leaves
- depth   9 has   36 nodes, of which   13 are leaves
- depth  10 has   46 nodes, of which   23 are leaves
- depth  11 has   46 nodes, of which   26 are leaves
- depth  12 has   40 nodes, of which   17 are leaves
- depth  13 has   46 nodes, of which   25 are leaves
- depth  14 has   42 nodes, of which   24 are leaves
- depth  15 has   36 nodes, of which   18 are leaves
- depth  16 has   36 nodes, of which   16 are leaves
- depth  17 has   40 nodes, of which   22 are leaves
- de

# Problem 2: Random forest

In [27]:
forest = sklearn.ensemble.RandomForestClassifier(
    criterion='gini', min_samples_split=2, random_state=101)

## 2A: Best Random Forest via grid search

Follow the instructions and using what you learn in 1B to finish this step.

This block might take 2-10 minutes. (Takes about 2 min on staff Macbook laptops.)

If yours runs significantly longer, try this out on Google Colab instead.

In [28]:
forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=125,
    criterion='gini',
    max_depth=15,
    min_samples_split=2,
    min_samples_leaf=1)


In [29]:
forest_hyperparameter_grid_by_name = dict(
    max_features=[3, 10, 33, 100, 333],
    max_depth=[16, 32],
    min_samples_leaf=[1],
    n_estimators=[125],
    random_state=[101],
    )

In [30]:
# TODO construct a GridSearchCV object like you did above.

forest_searcher = sklearn.model_selection.GridSearchCV(
    forest,
    forest_hyperparameter_grid_by_name,
    scoring='balanced_accuracy',
    cv=my_splitter,
    return_train_score=True,
    refit=False)

### Do the search!

In [32]:
#start_time_sec = time.time()
forest_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

### Build dataframe of results

In [33]:
#TODO
forest_results_df = pd.DataFrame(forest_searcher.cv_results_).copy()
print("Grid search of %3d configurations done after %6.1f sec" % (
    forest_results_df.shape[0], elapsed_time_sec))

Grid search of  10 configurations done after  321.4 sec


### Display search results

In [34]:
#TODO
pd.set_option('precision', 4)
forest_keys = ['param_max_depth', 'param_min_samples_leaf']
forest_results_df.sort_values(forest_keys, inplace=True)
forest_results_df[tree_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']]

,param_max_depth,param_min_samples_leaf,mean_train_score,mean_test_score,rank_test_score,mean_fit_time
0,16,1,0.9289,0.7582,10,1.3694
1,16,1,0.9340,0.8550,1,2.1859
2,16,1,0.9277,0.8400,5,5.5440
3,16,1,0.9217,0.8340,6,15.7287
4,16,1,0.8715,0.7942,9,42.4291
5,32,1,0.9800,0.8192,7,1.8146
6,32,1,0.9751,0.8534,2,3.6337
7,32,1,0.9720,0.8448,3,9.2643
8,32,1,0.9713,0.8405,4,22.9902
9,32,1,0.9527,0.8185,8,62.9188


### Build the best random forest using the best hyperparameters found in 2B and train it.

This is necessary so you have the specific best performing forest in your workspace.


In [35]:
#TODO
print("Printing a dict of the best hyperparameters")
print(forest_searcher.best_params_)

Printing a dict of the best hyperparameters
{'max_depth': 16, 'max_features': 10, 'min_samples_leaf': 1, 'n_estimators': 125, 'random_state': 101}


In [36]:
best_forest = forest.set_params(max_depth=16, max_features=10, min_samples_leaf=1, n_estimators=125, random_state=101) #TODO
best_forest.fit(x_tr_NF, y_tr_N)

RandomForestClassifier(max_depth=16, max_features=10, n_estimators=125,
                       random_state=101)

## 2B & Figure 2 : Feature Importances 

Access the **feature_importances_** attribute of your trained forest to get a score for each term in our vocabulary.

A higher value of this score indicates the feature is "more important".

In one panel of Figure 2, display a list of the top 10 vocabulary words of your best forest with highest feature importance.

In another panel of Figure 2, display a list of 10 randomly chosen terms that have close-to-zero feature importance (use anything with importance less than 0.00001).

### Figure 2.2
**Sample Output** (Feel free to print all words and organize them in any software)

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

In [37]:
features = best_forest.feature_importances_

np.asarray(vocab_list)[np.argsort(features)[:10]]


array(['money_to', 'while_you', 'usage', 'kinda', 'factori', 'few_months',
       'small_and', 'to_carry', 'it_arrived', 'concept_of'], dtype='<U21')

In [38]:
np.asarray(vocab_list)[np.argsort(features)[-10:]]

array(['disappoint', 'love', 'not_recommend', 'excel', 'bad', 'return',
       'your_money', 'great', 'poor', 'waste_of'], dtype='<U21')

# Problem 3: Comparison of models

### 3A Implementation **TODO**:

Selecting the best logistic regression with L1 penalty

Table 3 in Report

In [39]:
lasso = sklearn.linear_model.LogisticRegression(
    penalty='l1', solver='saga', random_state=101)

In [40]:
lasso_hyperparameter_grid_by_name = dict(
    C=np.logspace(-4, 4, 9),
    max_iter=[20, 40], # sneaky way to do "early stopping" 
                       # we'll take either iter 20 or iter 40 in training process, by best valid performance
    )

#### TODO Selecting the best logistic regression with L1 penalty
Hint: Follow 1B and 2A

In [53]:
lasso_searcher = sklearn.model_selection.GridSearchCV(
    lasso,
    lasso_hyperparameter_grid_by_name, 
    scoring='balanced_accuracy',
    cv=my_splitter,
    return_train_score=True,
    refit=False)

In [54]:
start_time_sec = time.time()
lasso_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

/Users/zoehsieh/opt/miniconda3/envs/comp135_2020f_env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zoehsieh/opt/miniconda3/envs/comp135_2020f_env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zoehsieh/opt/miniconda3/envs/comp135_2020f_env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zoehsieh/opt/miniconda3/envs/comp135_2020f_env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter wa

In [55]:
best_lasso = lasso.set_params(**lasso_searcher.best_params_)

best_lasso.fit(x_tr_NF, y_tr_N)

/Users/zoehsieh/opt/miniconda3/envs/comp135_2020f_env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(max_iter=20, penalty='l1', random_state=101, solver='saga')

### Table 3: Comparison of methods on the bag-of-words to sentiment classification task.

Please report balanced accuracy on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them in any software)

|**method**|**train**|**valid**|**test**|
|:-|:-:|:-:|:-:|
|L1-penalized LogisticRegression	|0.123	|0.456	|0.890|
|best RandomForest	|0.123	|0.456	|0.890|
|best Tree	|0.123	|0.456	|0.890|
|simple Tree	|0.123	|0.456	|0.890|

In [51]:
# best tree

# train
yhat_best_tr = best_tree.predict(x_tr_NF)
print(sklearn.metrics.accuracy_score(y_tr_N, yhat_best_tr))


# valid
yhat_best_va = best_tree.predict(x_va_TF)
print(sklearn.metrics.accuracy_score(y_va_T, yhat_best_va))

# test
yhat_best_te = best_tree.predict(x_te_TF)
print(sklearn.metrics.accuracy_score(y_te_T, yhat_best_te))


0.9981090450677592
0.7373737373737373
0.7263556116015133


In [50]:
# simple tree

# train
yhat_simple_tr = simple_tree.predict(x_tr_NF)
print(sklearn.metrics.accuracy_score(y_tr_N, yhat_simple_tr))


# valid
yhat_simple_va = simple_tree.predict(x_va_TF)
print(sklearn.metrics.accuracy_score(y_va_T, yhat_simple_va))

# test
yhat_simple_te = simple_tree.predict(x_te_TF)
print(sklearn.metrics.accuracy_score(y_te_T, yhat_simple_te))


0.6459186889379136
0.648989898989899
0.639344262295082


In [52]:
# forest

# train
yhat_forest_tr = best_forest.predict(x_tr_NF)
print(sklearn.metrics.accuracy_score(y_tr_N, yhat_best_tr))


# valid
yhat_forest_va = best_forest.predict(x_va_TF)
print(sklearn.metrics.accuracy_score(y_va_T, yhat_best_va))

# test
yhat_forest_te = best_forest.predict(x_te_TF)
print(sklearn.metrics.accuracy_score(y_te_T, yhat_best_te))


0.9981090450677592
0.7373737373737373
0.7263556116015133


In [56]:
# L1

# train
yhat_lasso_tr = best_lasso.predict(x_tr_NF)
print(sklearn.metrics.accuracy_score(y_tr_N, yhat_lasso_tr))


# valid
yhat_lasso_va = best_lasso.predict(x_va_TF)
print(sklearn.metrics.accuracy_score(y_va_T, yhat_lasso_va))

# test
yhat_lasso_te = best_forest.predict(x_te_TF)
print(sklearn.metrics.accuracy_score(y_te_T, yhat_lasso_te))


0.9383863851244879
0.8712121212121212
0.819672131147541
